# Advent of Code 2024

In [ ]:
import re
import itertools
import numpy as np
from tqdm import tqdm
from collections import defaultdict

## Day 1

In [ ]:
with open('data/day1.txt') as f:
    lines = f.read().splitlines() 

# Format into two lists
list1 = []
list2 = []
for line in lines:
    a, b = line.split("   ")
    list1.append(int(a))
    list2.append(int(b))

# Sort lists in ascending order
list1 = sorted(list1)
list2 = sorted(list2)

distance = 0
for a, b in zip(list1, list2):
    distance += abs(a - b)

print(f"Part 1 answer is {distance}")

similarity = 0
for a in list1:
    # Count occurences of a in list2 then multiply by a
    similarity += a*len([i for i, x in enumerate(list2) if x == a])

print(f"Part 2 answer is {similarity}")

## Day 2

In [ ]:
def is_safe(line):
    line = np.array([int(l) for l in line])
    diff = line[1:] - line[:-1]

    # disqualified if any diff is < 1 or > 3
    if (np.sum(np.abs(diff) > 3) == 0) and (np.sum(np.abs(diff) < 1) == 0):
        # disqualified if not all ascending or all descending
        if (np.sum(diff > 0) == 0) or (np.sum(diff < 0) == 0):
            return True
    else:
        return False

with open('data/day2.txt') as f:
    lines = f.read().splitlines()

lines = [l.split(" ") for l in lines]

# Part 1
safe = 0
for line in lines:
    if is_safe(line):
        safe += 1

print(f"Part 1 answer is {safe}")

# Part 2
safe = 0
for line in lines:
    for i in range(len(line)):
        # Just be lazy and make lots of copies of the list with a different item removed each time
        newline = line.copy()
        newline.pop(i)
        # If removing an item ever makes the list safe, it's safe
        if is_safe(newline):
            safe += 1
            break

print(f"Part 2 answer is {safe}")

## Day 3

In [ ]:
with open('data/day3.txt') as f:
    line = f.read()

# Extract all strings matching the pattern + capture groups
patterns = re.findall(r"mul\((\d{1,3}),(\d{1,3})\)", line)

total = 0
for pattern in patterns:
    total += int(pattern[0])*int(pattern[1])

print(f"Part 1 answer is {total}")

# Part 2
patterns = re.findall(r"(mul\((\d{1,3}),(\d{1,3})\)|do\(\)|don't\(\))", line)

do = True
total = 0
for pattern in patterns:
    if pattern[0] == "don't()":
        do = False
    elif pattern[0] == "do()":
        do = True
    else:
        if do:
            total += int(pattern[1])*int(pattern[2])

print(f"Part 2 answer is {total}")

## Day 4

In [ ]:
def count_string_in_array(line, string="XMAS"):
    # Turn it back into a string
    line = "".join(line)
    return len(re.findall(string, line))
    
with open('data/day4.txt') as f:
    lines = f.read().splitlines()

lines = [list(line) for line in lines]
lines = np.array(lines)

count = 0
# Find horizontal forwards and backwards
for line in lines:
    count += count_string_in_array(line)
    count += count_string_in_array(line[::-1])

# Vertical
for line in lines.transpose():
    count += count_string_in_array(line)
    count += count_string_in_array(line[::-1])

for i in range(-len(lines), len(lines)):
    # Diagonal one way
    line = lines.diagonal(i)
    count += count_string_in_array(line)
    count += count_string_in_array(line[::-1])
    # Diagonal the other way
    line = np.fliplr(lines).diagonal(i)
    count += count_string_in_array(line)
    count += count_string_in_array(line[::-1])

print(f"Part 1 answer is {count}")

# Find every 3x3 block
# Sliding filter convolution
count = 0
for i in range(lines.shape[0] - 2):
    for j in range(lines.shape[1] - 2):
        # Here is the 3 x 3 array
        grid = lines[i:i+3, j:j+3]
        diag1 = "".join(grid.diagonal())
        diag2 = "".join(np.fliplr(grid).diagonal())

        if (diag1 == "MAS") or (diag1 == "SAM"):
            if (diag2 == "MAS") or (diag2 == "SAM"):
                count += 1

print(f"Part 2 answer is {count}")

## Day 5

In [ ]:
def is_legal(update, rule_lookup):
    legal = True
    for i, u in enumerate(update):
        before_items = update[:i]
        after_items = update[i+1:]
    
        for item in before_items:
            # It's occured before u, so check that's allowed
            if item in rule_lookup[u]["after"]:
                legal = False
                break
        for item in after_items:
            # It's occured after u, so check that's allowed
            if item in rule_lookup[u]["before"]:
                legal = False
                break

    return legal

with open('data/day5.txt') as f:
    line = f.read()

rules, updates = line.split("\n\n")
rules = rules.split("\n")
updates = updates.split("\n")

rule_lookup = {}
for rule in rules:
    before, after = rule.split("|")
    before = int(before)
    after = int(after)
    
    if before not in rule_lookup:
        rule_lookup[before] = {
            "before": [],
            "after": []
        }
    rule_lookup[before]["after"].append(after)

    if after not in rule_lookup:
        rule_lookup[after] = {
            "before": [],
            "after": []
        }
    rule_lookup[after]["before"].append(before)

# I know this is a messy solution
answer = 0
for update in updates:
    update = update.split(",")
    update = [int(u) for u in update]

    # Check ordering correct
    if is_legal(update, rule_lookup):
        answer += update[int(len(update)/2)]

print(f"Part 1 answer is {answer}")


In [ ]:
def make_fix(update, rule_lookup):
    # Find a single broken rule and swap the two items involved
    for i, u in enumerate(update):
        before_items = update[:i]
        after_items = update[i+1:]
    
        for item in before_items:
            # It's occured before u, so check that's allowed
            if item in rule_lookup[u]["after"]:
                # if illegal, swap the two numbers
                update[update.index(item)] = u
                update[i] = item
                return False, update
        for item in after_items:
            # It's occured after u, so check that's allowed
            if item in rule_lookup[u]["before"]:
                # if illegal, swap the two numbers
                update[update.index(item)] = u
                update[i] = item
                return False, update

    # if nothing has been swapped, return None
    return True, update

answer = 0
for update in updates:
    update = update.split(",")
    update = [int(u) for u in update]

    # If the ordering is wrong...
    if not is_legal(update, rule_lookup):
        # Then go through update and swap the first two numbers which are wrong
        # Repeat until all numbers are right
        fixed = False
        while not fixed:
            fixed, update = make_fix(update, rule_lookup)

        answer += update[int(len(update)/2)]
        
print(f"Part 2 answer is {answer}")

## Day 6

In [ ]:
directions = {
    "^": np.array([-1, 0]),
    ">": np.array([0, 1]),
    "v": np.array([1, 0]),
    "<": np.array([0, -1])
}

directions_order = ["^", ">", "v", "<"]

def make_grid(lines):
    # Set up the grid
    grid = np.array([list(l) for l in lines])
    guard_location = np.array([np.where(grid == "^")[0][0], np.where(grid == "^")[1][0]])
    guard = Guard(location=guard_location, direction="^")
    return grid, guard


class Guard:
    def __init__(self, location, direction):
        self.location = location
        # In the format e.g. [-1, 0] means up one
        self.direction = direction

    def move(self, grid):
        # Where does the guard want to move to?
        target = self.location + directions[self.direction]

        # Check end condition
        if (target[0] < 0) or (target[0] >= grid.shape[0]) or (target[1] < 0) or (target[1] >= grid.shape[1]):
            return 1


        # If not end condition, check what is in the target square
        # If empty, move
        if grid[target[0], target[1]] not in ["#", "O"]:
            self.location = target
        else:
            self.direction = self.turn(self.direction)
        return 0

    def turn(self, direction):
        new_direction = directions_order[(directions_order.index(direction) + 1) % 4]
        return new_direction
        
with open('data/day6.txt') as f:
    lines = f.read().splitlines()

# Set up the grid
grid, guard = make_grid(lines)

state = 0
while state == 0:
    state = guard.move(grid)

    # Now update the grid
    grid[guard.location[0], guard.location[1]] = guard.direction

sumup = 0
for direction in directions:
    sumup += np.sum(grid == direction)
    
print(f"Part 1 answer is {sumup}")

# Set up the grid again
grid, guard = make_grid(lines)

# We try putting a blockage at every element of the grid
obstruction_locations = []
for i in tqdm(range(grid.shape[0])):
    for j in range(grid.shape[1]):
        # Set up the grid again
        test_grid, guard = make_grid(lines)
        test_grid[i, j] = "O"

        counter = 1
        old_sumup = 1
        resolved = False
        while not resolved:
            state = guard.move(test_grid)
            if state == 1:
                resolved = True
            else:
                # Check if we have reached a loop
                if test_grid[guard.location[0], guard.location[1]] == guard.direction:
                    resolved = True
                    obstruction_locations.append([i, j])
                # Update the grid
                test_grid[guard.location[0], guard.location[1]] = guard.direction

            # We have also reached a loop if we have not marked a new square in 10000 iterations
            # Know this is messy but I have an infinite loop bug I can't figure out..
            if counter % 10000 == 0:
                sumup = 0
                for direction in directions:
                    sumup += np.sum(test_grid == direction)
        
                if sumup == old_sumup:
                    resolved = True
                    obstruction_locations.append([i, j])
                old_sumup = sumup
            counter += 1
print(f"Part 2 answer is {len(obstruction_locations)}")

## Day 7

In [ ]:
with open('data/day7.txt') as f:
    lines = f.read().splitlines()

# Part 1
answer_list = []
number_list = []
for line in lines:
    answer, line = line.split(": ")
    numbers = line.split(" ")

    answer_list.append(int(answer))
    number_list.append([int(num) for num in numbers])
    
summed_answer = 0
for answer, numbers in zip(answer_list, number_list):
    # Get all possible arrangements of + and * given number of operators
    combinations = list(itertools.product(["*", "+"], repeat=(len(numbers)-1)))
    
    for operators in combinations:
        sumup = numbers[0]
        for num, operator in zip(numbers[1:], operators):
            if operator == "+":
                sumup += num
            elif operator == "*":
                sumup *= num

        if sumup == answer:
            summed_answer += answer
            break

print(f"Part 1 answer is {summed_answer}")

# Got lucky here because trivial to reuse part 1 solution
summed_answer = 0
for answer, numbers in zip(answer_list, number_list):
    combinations = list(itertools.product(["*", "+", "|"], repeat=(len(numbers)-1)))
    
    for operators in combinations:
        sumup = numbers[0]
        for num, operator in zip(numbers[1:], operators):
            if operator == "+":
                sumup += num
            elif operator == "*":
                sumup *= num
            elif operator == "|":
                sumup = int(str(sumup) + str(num))

        if sumup == answer:
            summed_answer += answer
            break

print(f"Part 2 answer is {summed_answer}")


## Day 8

In [ ]:
def get_coordinates_of_item_in_grid(grid, item):
    locations = np.where(grid==item)
    coordinates = []
    for x, y in zip(locations[0], locations[1]):
        coordinates.append([x, y])
    return np.array(coordinates)

def print_grid(grid):
    if grid.dtype == "float64":
        for line in grid:
            newline = ""
            for i in line:
                if i == 0:
                    newline += "."
                elif i == 1:
                    newline += "#"
            print(newline)
    else:
        for line in grid:
            print("".join(line))

def in_grid(grid, location):
    if (location[0] < 0) or (location[0] >= grid.shape[0]) or (location[1] < 0) or (location[1] >= grid.shape[1]):
        return False
    else:
        return True

with open('data/day8.txt') as f:
    lines = f.read().splitlines()

grid = np.array([list(l) for l in lines])
# Set up empty grid to store the antinodes in
antinode_grid = np.zeros([grid.shape[0], grid.shape[1]])

# What signals exist?
unique = list(np.unique(grid))
unique.remove(".")

for signal in unique:
    # I think we need the relationship between every possible pair
    # Relationship can be represented as a vector
    # We need starting point and direction
    signal_coordinates = get_coordinates_of_item_in_grid(grid, signal)

    # For each pair....
    antinode_vectors = []
    for i, coordinate1 in enumerate(signal_coordinates):
        for j, coordinate2 in enumerate(signal_coordinates):
            if i == j:
                pass
            else:
                position = coordinate1
                direction = coordinate2 - coordinate1
                antinode_vectors.append([position, direction])

    # Now as for where the antinode actually is, there are two potential antinodes per antinode_vector
    # These are position - direction and postion + direction*2
    potential_antinode_locations = []
    for vector in antinode_vectors:
        position = vector[0]
        direction = vector[1]
        potential_antinode_locations.append(position - direction)
        potential_antinode_locations.append(position + direction*2)

    # Now, some of these locations are not actually reasonable
    # This is because they are off the edge of the map
    for location in potential_antinode_locations:
        if in_grid(grid, location):
            antinode_grid[location[0], location[1]] = 1

print(f"Part 1 answer is {int(np.sum(antinode_grid))}")


In [ ]:
# Part 2 is very similar, only we do not stop adding antinodes

grid = np.array([list(l) for l in lines])
# Set up empty grid to store the antinodes in
antinode_grid = np.zeros([grid.shape[0], grid.shape[1]])

# What signals exist?
unique = list(np.unique(grid))
unique.remove(".")

for signal in unique:
    # I think we need the relationship between every possible pair
    # Relationship can be represented as a vector
    # We need starting point and direction
    signal_coordinates = get_coordinates_of_item_in_grid(grid, signal)

    # For each pair....
    antinode_vectors = []
    for i, coordinate1 in enumerate(signal_coordinates):
        for j, coordinate2 in enumerate(signal_coordinates):
            if i == j:
                pass
            else:
                position = coordinate1
                direction = coordinate2 - coordinate1
                antinode_vectors.append([position, direction])

    # Now as for where the antinode actually is, there are two potential antinodes per antinode_vector
    # These are position - direction and postion + direction*2
    potential_antinode_locations = []
    for vector in antinode_vectors:
        position = vector[0]
        direction = vector[1]

        # Every antenna is an antinode
        antinode_grid[position[0], position[1]] = 1
        
        # Deal with backward antinodes
        # Add backward antinodes until illegal
        inside_grid = True
        n = 1
        while inside_grid:
            location = position - n*direction
            if in_grid(grid, location):
                antinode_grid[location[0], location[1]] = 1
                n += 1
            else:
                inside_grid = False

        # Deal with forward antinodes
        n = 2
        while inside_grid:
            location = position + n*direction
            if in_grid(grid, location):
                antinode_grid[location[0], location[1]] = 1
                n += 1
            else:
                inside_grid = False


print(f"Part 2 answer is {int(np.sum(antinode_grid))}")